In [1]:
import pandas as pd 
import os

In [2]:
def RSI4 (stock):
    
    Hist_data = pd.read_csv(stock)
    Company = ((os.path.basename(stock)).split(".csv")[0])  # Name of the company
    # This list holds the closing prices of a stock
    prices = []
    c = 0
    # Add the closing prices to the prices list and make sure we start at greater than 2 dollars to reduce outlier calculations.
    while c < len(Hist_data):
        if Hist_data.iloc[c,4] > float(2.00):  # Check that the closing price for this day is greater than $2.00
            prices.append(Hist_data.iloc[c,4])
        c += 1
    # prices_df = pd.DataFrame(prices)  # Make a dataframe from the prices list
    i = 0
    #print(prices)
    upPrices=[]
    downPrices=[]
    #  Loop to hold up and down price movements
    while i < len(prices):
        if i == 0:
            upPrices.append(0)
            downPrices.append(0)
        else:
            if (prices[i]-prices[i-1])>0:
                upPrices.append(prices[i]-prices[i-1])
                downPrices.append(0)
            else:
                downPrices.append(prices[i]-prices[i-1])
                upPrices.append(0)
        i += 1
    x = 0
    avg_gain = []
    avg_loss = []
    #  Loop to calculate the average gain and loss
    while x < len(upPrices):
        if x <15:
            avg_gain.append(0)
            avg_loss.append(0)
        else:
            sumGain = 0
            sumLoss = 0
            y = x-14
            while y<=x:
                sumGain += upPrices[y]
                sumLoss += downPrices[y]
                y += 1
            avg_gain.append(sumGain/14)
            avg_loss.append(abs(sumLoss/14))
        x += 1
    p = 0
    RS = []
    RSI = []
    #  Loop to calculate RSI and RS
    while p < len(prices):
        if p <15:
            RS.append(0)
            RSI.append(0)
        else:
            RSvalue = (avg_gain[p]/avg_loss[p])
            RS.append(RSvalue)
            RSI.append(100 - (100/(1+RSvalue)))
        p+=1
    #  Creates the csv for each stock's RSI and price movements
    df_dict = {
        'Prices' : prices,
        'upPrices' : upPrices,
        'downPrices' : downPrices,
        'AvgGain' : avg_gain,
        'AvgLoss' : avg_loss,
        'RS' : RS,
        'RSI' : RSI
    }
    df = pd.DataFrame(df_dict, `szaxcolumns = ['Prices', 'upPrices', 'downPrices', 'AvgGain','AvgLoss', 'RS', "RSI"])
    df.to_csv(Company+"_RSI.csv", index = False)
    return RSI, prices

In [3]:
RSI, prices = RSI4("final_data.csv")

In [4]:
df = pd.read_csv("final_data.csv")

In [5]:
df

,date,open,high,low,close
0,1613055240000,47800.21,47858.35,47795.86,47821.59
1,1613055300000,47824.01,47883.73,47814.77,47857.40
2,1613055360000,47857.41,47890.16,47816.93,47846.43
3,1613055420000,47846.42,47874.99,47834.30,47868.56
4,1613055480000,47868.56,47871.85,47814.89,47837.24
...,...,...,...,...,...
41503,1615550820000,56538.80,56556.00,56469.21,56500.00
41504,1615550880000,56500.00,56714.00,56500.00,56692.79
41505,1615550940000,56692.79,56778.21,56658.80,56715.36
41506,1615550940000,56692.79,56778.21,56658.80,56746.06


In [6]:
len(prices)

41508

In [7]:
prices[19]

47669.68

In [8]:
RSI[19]

41.81260261121111

In [9]:
Compare_Stocks = pd.DataFrame(columns=["Company", "Days_Observed", "Crosses", "True_Positive", "False_Positive", "True_Negative", "False_Negative", "Sensitivity", 
"Specificity", "Accuracy", "TPR", "FPR"])

In [10]:
#  Code to test the accuracy of the RSI at predicting stock prices
Days_Observed = 14
Crosses = 0
nothing = 0
True_Positive = 0
False_Positive = 0
True_Negative = 0
False_Negative = 0
Sensitivity = 0
Specificity = 0
Accuracy = 0
print(len(prices))
while Days_Observed < len(prices)-14:
    
    if RSI[Days_Observed] <= 20:
        #print("--------------------------",RSI[Days_Observed],'------------')
        #print
        if ((prices[Days_Observed + 1] + prices[Days_Observed + 2] + prices[Days_Observed + 3] + prices[Days_Observed + 4] + prices[Days_Observed + 5])/5) > prices[Days_Observed]:
            True_Positive += 1
#             print("-------------", prices[Days_Observed],'-------------------')
#             print("-------", Days_Observed)
        else:
            False_Negative += 1
        Crosses += 1
    elif RSI[Days_Observed] >= 80:
#         print("--------------------------",RSI[Days_Observed],'------------')
        if ((prices[Days_Observed + 1] + prices[Days_Observed + 2] + prices[Days_Observed + 3] + prices[Days_Observed + 4] + prices[Days_Observed + 5])/5) <= prices[Days_Observed]:
            True_Negative += 1
#             print("-------------", prices[Days_Observed],'-------------------')
#             print("-------", Days_Observed)
        else:
            False_Positive += 1
#             print("-------------", prices[Days_Observed],'-------------------')
#             print("-------", Days_Observed)
        Crosses += 1
    else:
        #Do nothing
        nothing+=1
    Days_Observed += 1
    # while Days_Observed<len(prices)-5:

    #     Days_Observed += 1
    try:
        Sensitivity = (True_Positive / (True_Positive + False_Negative)) # Calculate sensitivity
    except ZeroDivisionError:  # Catch the divide by zero error
        Sensitivity = 0
    try:
        Specificity = (True_Negative / (True_Negative + False_Positive)) # Calculate specificity
    except ZeroDivisionError:
        Specificity = 0
    try:
        Accuracy = (True_Positive + True_Negative) / (True_Negative + True_Positive + False_Positive + False_Negative) # Calculate accuracy
    except ZeroDivisionError:
        Accuracy = 0
    TPR = Sensitivity  # Calculate the true positive rate
    FPR = 1 - Specificity  # Calculate the false positive rate
    # Create a row to add to the compare_stocks
    add_row = {'Company' : "BTC", 'Days_Observed' : Days_Observed, 'Crosses' : Crosses, 'True_Positive' : True_Positive, 'False_Positive' : False_Positive, 
    'True_Negative' : True_Negative, 'False_Negative' : False_Negative, 'Sensitivity' : Sensitivity, 'Specificity' : Specificity, 'Accuracy' : Accuracy, 'TPR' : TPR, 'FPR' : FPR}
    Compare_Stocks = Compare_Stocks.append(add_row, ignore_index = True) # Add the analysis on the stock to the existing Compare_Stocks dataframe
Compare_Stocks.to_csv("prediction.csv", index = False)  # Save the compiled data on each stock to a csv

41508


In [11]:
df = pd.read_csv("prediction.csv")

In [12]:
df

,Company,Days_Observed,Crosses,True_Positive,False_Positive,True_Negative,False_Negative,Sensitivity,Specificity,Accuracy,TPR,FPR
0,BTC,15,1,0,0,0,1,0.00000,0.000000,0.000000,0.00000,1.000000
1,BTC,16,1,0,0,0,1,0.00000,0.000000,0.000000,0.00000,1.000000
2,BTC,17,1,0,0,0,1,0.00000,0.000000,0.000000,0.00000,1.000000
3,BTC,18,1,0,0,0,1,0.00000,0.000000,0.000000,0.00000,1.000000
4,BTC,19,1,0,0,0,1,0.00000,0.000000,0.000000,0.00000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
41475,BTC,41490,1552,385,404,499,264,0.59322,0.552602,0.569588,0.59322,0.447398
41476,BTC,41491,1552,385,404,499,264,0.59322,0.552602,0.569588,0.59322,0.447398
41477,BTC,41492,1552,385,404,499,264,0.59322,0.552602,0.569588,0.59322,0.447398
41478,BTC,41493,1552,385,404,499,264,0.59322,0.552602,0.569588,0.59322,0.447398
